In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LearningRateScheduler
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tensorflow.keras import layers
from tensorflow.keras import models
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

In [5]:
data=pd.read_csv("../data/data_proccesed_50.csv")
X = data.iloc[: ,:-1].to_numpy()
Y = data['labels'].to_numpy()

X = X.reshape(X.shape[0], X.shape[1], 1)


x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [6]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

In [22]:
x_train.shape

(459104, 50, 1)

In [21]:
x_test.shape

(114776, 50, 1)

In [26]:
def custom_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.02))
scheduler = LearningRateScheduler(custom_scheduler)

model_save = ModelCheckpoint(r"../model/best_model.h5",
                              save_best_only=True,
                             mode='max',
                             monitor='val_accuracy',
                             initial_value_threshold=0.9,
                             verbose=1)

normalizer = layers.Normalization()
normalizer.adapt(x_train)

model_lstm = models.Sequential([
    layers.Input(shape=(x_train.shape[1], x_train.shape[2])), 
    normalizer,
    layers.LSTM(64, activation="tanh"),
    layers.Dropout(0.3),
    layers.Dense(1, activation="sigmoid"),
])


model_lstm.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['accuracy'],
)

In [27]:
history_lstm=model_lstm.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test), callbacks=[scheduler,model_save])

Epoch 1/10
7174/7174 ━━━━━━━━━━━━━━━━━━━━ 62s 8ms/step - accuracy: 0.9778 - loss: 0.0183 - val_accuracy: 0.9919 - val_loss: 0.0069
Epoch 2/10
7174/7174 ━━━━━━━━━━━━━━━━━━━━ 69s 10ms/step - accuracy: 0.9954 - loss: 0.0043 - val_accuracy: 0.9979 - val_loss: 0.0020
Epoch 3/10
7174/7174 ━━━━━━━━━━━━━━━━━━━━ 72s 10ms/step - accuracy: 0.9970 - loss: 0.0027 - val_accuracy: 0.9978 - val_loss: 0.0021
Epoch 4/10
7174/7174 ━━━━━━━━━━━━━━━━━━━━ 72s 10ms/step - accuracy: 0.9978 - loss: 0.0021 - val_accuracy: 0.9988 - val_loss: 0.0010
Epoch 5/10
7174/7174 ━━━━━━━━━━━━━━━━━━━━ 72s 10ms/step - accuracy: 0.9984 - loss: 0.0015 - val_accuracy: 0.9986 - val_loss: 0.0012
Epoch 6/10
7174/7174 ━━━━━━━━━━━━━━━━━━━━ 71s 10ms/step - accuracy: 0.9986 - loss: 0.0013 - val_accuracy: 0.9990 - val_loss: 8.9234e-04
Epoch 7/10
7174/7174 ━━━━━━━━━━━━━━━━━━━━ 71s 10ms/step - accuracy: 0.9989 - loss: 0.0010 - val_accuracy: 0.9991 - val_loss: 0.0010
Epoch 8/10
7174/7174 ━━━━━━━━━━━━━━━━━━━━ 71s 10ms/step - accuracy: 0.998

In [30]:
model_lstm.save(r"../model/model_99.h5")

In [8]:
model_lstm = tf.keras.models.load_model('model.h5')

In [ ]:
metrics = history_lstm.history
plt.plot(history_lstm.epoch, metrics['accuracy'], metrics['val_accuracy'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [9]:
model_lstm.evaluate(x_test,y_test)

3587/3587 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.8303 - loss: 0.1608


[0.16016033291816711, 0.8311493396759033]